<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/autogluon_M1_next_day_graph_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon

     |████████████████████████████████| 56 kB 1.3 MB/s 
     |████████████████████████████████| 273 kB 48.6 MB/s 
     |████████████████████████████████| 352 kB 36.7 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 210 kB 59.9 MB/s 
     |████████████████████████████████| 131 kB 55.7 MB/s 
     |████████████████████████████████| 802 kB 52.8 MB/s 
     |████████████████████████████████| 4.2 MB 44.2 MB/s 
     |████████████████████████████████| 27.4 MB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 57.1 MB/s 
     |████████████████████████████████| 119 kB 53.8 MB/s 
     |████████████████████████████████| 3.0 MB 33.2 MB/s 
     |████████████████████████████████| 296 kB 47.5 MB/s 
     |████████████████████████████████| 189 kB 40.6 MB/s 
     |████████████████████████████████| 166.7 MB 14 kB/s 
     |████████████████████████████████| 67.3 MB 12 kB/s 
     |█████████████

In [3]:
!pip install --upgrade mxnet

In [42]:
#designed for Jupyter/kaggle/colab
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing

#autogluon
from autogluon.tabular import TabularDataset, TabularPredictor  

In [43]:
 #MT4 csv 
df=pd.read_csv('EURUSD1.csv', names=['d','t','eu_o','eu_h','eu_l','eu_c','eu_v'])


# df2=pd.read_csv('GBPUSD1440.csv', names=['d','t','gu_o','gu_h','gu_l','gu_c','gu_v'])
# df2 = df2.tail(2000)

# df3=pd.read_csv('USDCAD1440.csv', names=['d','t','uc_o','uc_h','uc_l','uc_c','uc_v'])
# df3 = df3.tail(2000)

# df4=pd.read_csv('USDCHF1440.csv', names=['d','t','uf_o','uf_h','uf_l','uf_c','uf_v'])
# df4 = df4.tail(2000)

# df5=pd.read_csv('USDJPY1440.csv', names=['d','t','uj_o','uj_h','uj_l','uj_c','uj_v'])
# df5 = df5.tail(2000)

# df6=pd.read_csv('USDSEK1440.csv', names=['d','t','us_o','us_h','us_l','us_c','us_v'])
# df6 = df6.tail(2000)

# #using merge
# df_m_1 = df1.merge(df2, on='d')
# df_m_2 = df_m_1.merge(df3, on='d')
# df_m_3 = df_m_2.merge(df4, on='d')
# df_m_4 = df_m_3.merge(df5, on='d')
# df_m_5 = df_m_4.merge(df6, on='d')
# df_m_5 = df_m_5.drop(columns='t_x')
# df_m_5 = df_m_5.drop(columns='t_y')

# pd.set_option('display.max_columns', None)
# print(df_m_5) 

In [44]:
df=pd.DataFrame(data=df.tail(50000))
df

,d,t,eu_o,eu_h,eu_l,eu_c,eu_v
15000,2021.11.03,19:46,1.15788,1.15792,1.15777,1.15781,42
15001,2021.11.03,19:47,1.15780,1.15787,1.15780,1.15787,67
15002,2021.11.03,19:48,1.15787,1.15793,1.15785,1.15792,51
15003,2021.11.03,19:49,1.15790,1.15807,1.15790,1.15801,92
15004,2021.11.03,19:50,1.15800,1.15817,1.15800,1.15807,82
...,...,...,...,...,...,...,...
64995,2021.12.24,23:50,1.13210,1.13210,1.13055,1.13182,17
64996,2021.12.24,23:51,1.13181,1.13200,1.13181,1.13181,13
64997,2021.12.24,23:52,1.13179,1.13198,1.13063,1.13068,14
64998,2021.12.24,23:53,1.13071,1.13125,1.13036,1.13105,19


In [45]:
df = df.drop(['eu_o','eu_h','eu_l','eu_v'], axis=1)

In [46]:
pd.set_option('display.max_columns', 100)
df 


,d,t,eu_c
15000,2021.11.03,19:46,1.15781
15001,2021.11.03,19:47,1.15787
15002,2021.11.03,19:48,1.15792
15003,2021.11.03,19:49,1.15801
15004,2021.11.03,19:50,1.15807
...,...,...,...
64995,2021.12.24,23:50,1.13182
64996,2021.12.24,23:51,1.13181
64997,2021.12.24,23:52,1.13068
64998,2021.12.24,23:53,1.13105


In [47]:
df = df.drop(['d','t'], axis=1)

In [48]:
df4 = pd.DataFrame(data=df.iloc[-5760:-4320])
df4['row_num']=np.arange(len(df4))
df4.set_index('row_num',inplace=True)
df4 = df4.rename(columns={'eu_c':'4'})

print(df4)

df3 = pd.DataFrame(data=df.iloc[-4320:-2880])
df3['row_num']=np.arange(len(df3))
df3.set_index('row_num',inplace=True)
df3 = df3.rename(columns={'eu_c':'3'})

print(df3)

df2 = pd.DataFrame(data=df.iloc[-2880:-1440])
df2['row_num']=np.arange(len(df2))
df2.set_index('row_num',inplace=True)
df2 = df2.rename(columns={'eu_c':'2'})

print(df2)

df1 = pd.DataFrame(data=df.iloc[-1440:])
df1['row_num']=np.arange(len(df1))
df1.set_index('row_num',inplace=True)
df1 = df1.rename(columns={'eu_c':'1'})
print(df1)


               4
row_num         
0        1.12763
1        1.12754
2        1.12764
3        1.12763
4        1.12756
...          ...
1435     1.12808
1436     1.12808
1437     1.12812
1438     1.12811
1439     1.12811

[1440 rows x 1 columns]
               3
row_num         
0        1.12795
1        1.12800
2        1.12800
3        1.12800
4        1.12806
...          ...
1435     1.13327
1436     1.13325
1437     1.13325
1438     1.13322
1439     1.13323

[1440 rows x 1 columns]
               2
row_num         
0        1.13332
1        1.13323
2        1.13317
3        1.13305
4        1.13308
...          ...
1435     1.13270
1436     1.13271
1437     1.13271
1438     1.13270
1439     1.13276

[1440 rows x 1 columns]
               1
row_num         
0        1.13274
1        1.13278
2        1.13270
3        1.13280
4        1.13281
...          ...
1435     1.13182
1436     1.13181
1437     1.13068
1438     1.13105
1439     1.13133

[1440 rows x 1 columns]


In [49]:
df_post = pd.concat([df1,df2,df3,df4], axis=1)
df_post = pd.DataFrame(data=df_post)
df_post


,1,2,3,4
row_num,,,,
0,1.13274,1.13332,1.12795,1.12763
1,1.13278,1.13323,1.12800,1.12754
2,1.13270,1.13317,1.12800,1.12764
3,1.13280,1.13305,1.12800,1.12763
4,1.13281,1.13308,1.12806,1.12756
...,...,...,...,...
1435,1.13182,1.13270,1.13327,1.12808
1436,1.13181,1.13271,1.13325,1.12808
1437,1.13068,1.13271,1.13325,1.12812


In [52]:
df_test = pd.concat([df2,df3,df4], axis=1)
df_test = pd.DataFrame(data=df_test)
df_test

,2,3,4
row_num,,,
0,1.13332,1.12795,1.12763
1,1.13323,1.12800,1.12754
2,1.13317,1.12800,1.12764
3,1.13305,1.12800,1.12763
4,1.13308,1.12806,1.12756
...,...,...,...
1435,1.13270,1.13327,1.12808
1436,1.13271,1.13325,1.12808
1437,1.13271,1.13325,1.12812


In [ ]:

# #Features
# #1 day ago (features)
# df_m_5['eu_High_last_day1'] = df_m_5['eu_h'].shift(1)
# df_m_5['eu_Low_last_day1'] = df_m_5['eu_l'].shift(1)
# df_m_5['eu_Close_last_day1'] = df_m_5['eu_c'].shift(1) 
# #2 days ago
# df_m_5['eu_High_last_day2'] = df_m_5['eu_h'].shift(2)
# df_m_5['eu_Low_last_day2'] = df_m_5['eu_l'].shift(2)
# df_m_5['eu_Close_last_day2'] = df_m_5['eu_c'].shift(2) 
# #3 days ago
# df_m_5['eu_High_last_day3'] = df_m_5['eu_h'].shift(3)
# df_m_5['eu_Low_last_day3'] = df_m_5['eu_l'].shift(3)
# df_m_5['eu_Close_last_day3'] = df_m_5['eu_c'].shift(3) 
# #4 days ago 
# df_m_5['eu_High_last_day4'] = df_m_5['eu_h'].shift(4)
# df_m_5['eu_Low_last_day4'] = df_m_5['eu_l'].shift(4)
# df_m_5['eu_Close_last_day4'] = df_m_5['eu_c'].shift(4) 
# #5 days ago
# df_m_5['eu_High_last_day5'] = df_m_5['eu_h'].shift(5)
# df_m_5['eu_Low_last_day5'] = df_m_5['eu_l'].shift(5)
# df_m_5['eu_Close_last_day5'] = df_m_5['eu_c'].shift(5) 
# #6 day ago (features)
# df_m_5['eu_High_last_day6'] = df_m_5['eu_h'].shift(6)
# df_m_5['eu_Low_last_day6'] = df_m_5['eu_l'].shift(6)
# df_m_5['eu_Close_last_day6'] = df_m_5['eu_c'].shift(6) 
# #7 days ago
# df_m_5['eu_High_last_day7'] = df_m_5['eu_h'].shift(7)
# df_m_5['eu_Low_last_day7'] = df_m_5['eu_l'].shift(7)
# df_m_5['eu_Close_last_day7'] = df_m_5['eu_c'].shift(7) 
# #8 days ago
# df_m_5['eu_High_last_day8'] = df_m_5['eu_h'].shift(8)
# df_m_5['eu_Low_last_day8'] = df_m_5['eu_l'].shift(8)
# df_m_5['eu_Close_last_day8'] = df_m_5['eu_c'].shift(8) 
# #9 days ago 
# df_m_5['eu_High_last_day9'] = df_m_5['eu_h'].shift(9)
# df_m_5['eu_Low_last_day9'] = df_m_5['eu_l'].shift(9)
# df_m_5['eu_Close_last_day9'] = df_m_5['eu_c'].shift(9) 
# #10 days ago
# df_m_5['eu_High_last_day10'] = df_m_5['eu_h'].shift(10)
# df_m_5['eu_Low_last_day10'] = df_m_5['eu_l'].shift(10)
# df_m_5['eu_Close_last_day10'] = df_m_5['eu_c'].shift(10) 

# #11 day ago (features)
# df_m_5['eu_High_last_day11'] = df_m_5['eu_h'].shift(11)
# df_m_5['eu_Low_last_day11'] = df_m_5['eu_l'].shift(11)
# df_m_5['eu_Close_last_day11'] = df_m_5['eu_c'].shift(11) 
# #12 days ago
# df_m_5['eu_High_last_day12'] = df_m_5['eu_h'].shift(12)
# df_m_5['eu_Low_last_day12'] = df_m_5['eu_l'].shift(12)
# df_m_5['eu_Close_last_day12'] = df_m_5['eu_c'].shift(12) 
# #13 days ago
# df_m_5['eu_High_last_day13'] = df_m_5['eu_h'].shift(13)
# df_m_5['eu_Low_last_day13'] = df_m_5['eu_l'].shift(13)
# df_m_5['eu_Close_last_day13'] = df_m_5['eu_c'].shift(13) 
# #14 days ago 
# df_m_5['eu_High_last_day14'] = df_m_5['eu_h'].shift(14)
# df_m_5['eu_Low_last_day14'] = df_m_5['eu_l'].shift(14)
# df_m_5['eu_Close_last_day14'] = df_m_5['eu_c'].shift(14) 
# #15 days ago
# df_m_5['eu_High_last_day15'] = df_m_5['eu_h'].shift(15)
# df_m_5['eu_Low_last_day15'] = df_m_5['eu_l'].shift(15)
# df_m_5['eu_Close_last_day15'] = df_m_5['eu_c'].shift(15) 
# #16 day ago (features)
# df_m_5['eu_High_last_day16'] = df_m_5['eu_h'].shift(16)
# df_m_5['eu_Low_last_day16'] = df_m_5['eu_l'].shift(16)
# df_m_5['eu_Close_last_day16'] = df_m_5['eu_c'].shift(16) 
# #17 days ago
# df_m_5['eu_High_last_day17'] = df_m_5['eu_h'].shift(17)
# df_m_5['eu_Low_last_day17'] = df_m_5['eu_l'].shift(17)
# df_m_5['eu_Close_last_day17'] = df_m_5['eu_c'].shift(17) 
# #18 days ago
# df_m_5['eu_High_last_day18'] = df_m_5['eu_h'].shift(18)
# df_m_5['eu_Low_last_day18'] = df_m_5['eu_l'].shift(18)
# df_m_5['eu_Close_last_day18'] = df_m_5['eu_c'].shift(18) 
# #19 days ago 
# df_m_5['eu_High_last_day19'] = df_m_5['eu_h'].shift(19)
# df_m_5['eu_Low_last_day19'] = df_m_5['eu_l'].shift(19)
# df_m_5['eu_Close_last_day19'] = df_m_5['eu_c'].shift(19) 
# #20 days ago
# df_m_5['eu_High_last_day20'] = df_m_5['eu_h'].shift(20)
# df_m_5['eu_Low_last_day20'] = df_m_5['eu_l'].shift(20)
# df_m_5['eu_Close_last_day20'] = df_m_5['eu_c'].shift(20) 

# #21 day ago (features)
# df_m_5['eu_High_last_day21'] = df_m_5['eu_h'].shift(21)
# df_m_5['eu_Low_last_day21'] = df_m_5['eu_l'].shift(21)
# df_m_5['eu_Close_last_day21'] = df_m_5['eu_c'].shift(21) 
# #22 days ago
# df_m_5['eu_High_last_day22'] = df_m_5['eu_h'].shift(22)
# df_m_5['eu_Low_last_day22'] = df_m_5['eu_l'].shift(22)
# df_m_5['eu_Close_last_day22'] = df_m_5['eu_c'].shift(22) 
# #23 days ago
# df_m_5['eu_High_last_day23'] = df_m_5['eu_h'].shift(23)
# df_m_5['eu_Low_last_day23'] = df_m_5['eu_l'].shift(23)
# df_m_5['eu_Close_last_day23'] = df_m_5['eu_c'].shift(23) 
# #24 days ago 
# df_m_5['eu_High_last_day24'] = df_m_5['eu_h'].shift(24)
# df_m_5['eu_Low_last_day24'] = df_m_5['eu_l'].shift(24)
# df_m_5['eu_Close_last_day24'] = df_m_5['eu_c'].shift(24) 
# #25 days ago
# df_m_5['eu_High_last_day25'] = df_m_5['eu_h'].shift(25)
# df_m_5['eu_Low_last_day25'] = df_m_5['eu_l'].shift(25)
# df_m_5['eu_Close_last_day25'] = df_m_5['eu_c'].shift(25) 
# #26 day ago (features)
# df_m_5['eu_High_last_day26'] = df_m_5['eu_h'].shift(26)
# df_m_5['eu_Low_last_day26'] = df_m_5['eu_l'].shift(26)
# df_m_5['eu_Close_last_day26'] = df_m_5['eu_c'].shift(26) 
# #27 days ago
# df_m_5['eu_High_last_day27'] = df_m_5['eu_h'].shift(27)
# df_m_5['eu_Low_last_day27'] = df_m_5['eu_l'].shift(27)
# df_m_5['eu_Close_last_day27'] = df_m_5['eu_c'].shift(27) 
# #28 days ago
# df_m_5['eu_High_last_day28'] = df_m_5['eu_h'].shift(28)
# df_m_5['eu_Low_last_day28'] = df_m_5['eu_l'].shift(28)
# df_m_5['eu_Close_last_day28'] = df_m_5['eu_c'].shift(28) 
# #29 days ago 
# df_m_5['eu_High_last_day29'] = df_m_5['eu_h'].shift(29)
# df_m_5['eu_Low_last_day29'] = df_m_5['eu_l'].shift(29)
# df_m_5['eu_Close_last_day29'] = df_m_5['eu_c'].shift(29) 
# #30 days ago
# df_m_5['eu_High_last_day30'] = df_m_5['eu_h'].shift(30)
# df_m_5['eu_Low_last_day30'] = df_m_5['eu_l'].shift(30)
# df_m_5['eu_Close_last_day30'] = df_m_5['eu_c'].shift(30) 

# #Target
# #1 day forward 
# df_m_5['eu_High_next_day1'] = df_m_5['eu_h'].shift(-1)
# df_m_5['eu_Low_next_day1'] = df_m_5['eu_l'].shift(-1)
# df_m_5['eu_Close_next_day1'] = df_m_5['eu_c'].shift(-1) 

# #drop null and useless columns
# df_m_5 = df_m_5.dropna()
# df_m_5 = df_m_5.drop(['d','t','eu_o','eu_v'], axis=1)



In [20]:
extra_clone = pd.DataFrame(data=df_post)
print(extra_clone)


               1        2        3        4
row_num                                    
0        1.13274  1.13332  1.12795  1.12763
1        1.13278  1.13323  1.12800  1.12754
2        1.13270  1.13317  1.12800  1.12764
3        1.13280  1.13305  1.12800  1.12763
4        1.13281  1.13308  1.12806  1.12756
...          ...      ...      ...      ...
1435     1.13182  1.13270  1.13327  1.12808
1436     1.13181  1.13271  1.13325  1.12808
1437     1.13068  1.13271  1.13325  1.12812
1438     1.13105  1.13270  1.13322  1.12811
1439     1.13133  1.13276  1.13323  1.12811

[1440 rows x 4 columns]


In [21]:
extra_clone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1440 entries, 0 to 1439
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       1440 non-null   float64
 1   2       1440 non-null   float64
 2   3       1440 non-null   float64
 3   4       1440 non-null   float64
dtypes: float64(4)
memory usage: 56.2 KB


In [36]:
# #extract yesterday's row to predict today's row
# #check if needs to be pd.DataFrame
# last_row = pd.DataFrame(data=extra_clone.iloc[-10:,np.r_[0:93]])
# last_row

In [35]:
# df_m_5 = pd.DataFrame(data=df_m_5.tail(8000))
# df_m_5

In [23]:
#create initial train and test data as pd.DF
#final test DF is last_row
df_post_train = pd.DataFrame(data=df_post.iloc[0:700,:])
df_post_test = pd.DataFrame(data=df_post.iloc[700:1400,:])
print(df_post_train)
print(df_post_test)

               1        2        3        4
row_num                                    
0        1.13274  1.13332  1.12795  1.12763
1        1.13278  1.13323  1.12800  1.12754
2        1.13270  1.13317  1.12800  1.12764
3        1.13280  1.13305  1.12800  1.12763
4        1.13281  1.13308  1.12806  1.12756
...          ...      ...      ...      ...
695      1.13403  1.13164  1.12733  1.12999
696      1.13401  1.13177  1.12728  1.13001
697      1.13395  1.13172  1.12713  1.13001
698      1.13392  1.13164  1.12719  1.12999
699      1.13401  1.13158  1.12733  1.12998

[700 rows x 4 columns]
               1        2        3        4
row_num                                    
700      1.13404  1.13153  1.12729  1.12994
701      1.13403  1.13144  1.12732  1.13002
702      1.13404  1.13165  1.12730  1.13012
703      1.13404  1.13171  1.12724  1.12996
704      1.13407  1.13172  1.12719  1.12978
...          ...      ...      ...      ...
1395     1.13166  1.13340  1.13342  1.12812
1396    

In [ ]:
# #new column: high vs low
# df_m_5['eu_h_or_l'] = ((df_m_5.eu_h - df_m_5.eu_o) > (df_m_5.eu_o - df_m_5.eu_l)) 
# df_m_5.eu_h_or_l = df_m_5.eu_h_or_l.replace({True:1,False:0})

# #new column: close vs yesterday
# df_m_5['eu_c_vs_c'] = (df_m_5.eu_c > df_m_5.eu_o) 
# df_m_5.eu_c_vs_c = df_m_5.eu_c_vs_c.replace({True:1,False:0})

# #new column: shift tomorrow to today
# df_m_5['eu_High_next_day'] = df_m_5['eu_h'].shift(-1)
# df_m_5['eu_Low_next_day'] = df_m_5['eu_l'].shift(-1)
# df_m_5['eu_Close_next_day'] = df_m_5['eu_c'].shift(-1)
# df_m_5 = df_m_5.dropna()


In [24]:

print(df_post_train.isna().sum())
print(df_post_test.isna().sum())

1    0
2    0
3    0
4    0
dtype: int64
1    0
2    0
3    0
4    0
dtype: int64


In [25]:
print(df_post_train.info())
print(df_post_test.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 0 to 699
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       700 non-null    float64
 1   2       700 non-null    float64
 2   3       700 non-null    float64
 3   4       700 non-null    float64
dtypes: float64(4)
memory usage: 27.3 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 700 to 1399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       700 non-null    float64
 1   2       700 non-null    float64
 2   3       700 non-null    float64
 3   4       700 non-null    float64
dtypes: float64(4)
memory usage: 27.3 KB
None


In [26]:
train_data = df_post_train
train_data=train_data.sample(n=500, replace=True)
train_data.head()

,1,2,3,4
row_num,,,,
117,1.13262,1.13257,1.12852,1.12774
118,1.13259,1.13258,1.12854,1.12768
149,1.13261,1.13310,1.12862,1.12755
599,1.13311,1.13321,1.12658,1.12795
403,1.13289,1.13383,1.12724,1.12818


In [29]:
#predict new interval
label_new_interval = '4' 
print("Summary of class variable: \n", train_data[label_new_interval].describe())

Summary of class variable: 
 count    500.000000
mean       1.128073
std        0.000576
min        1.127360
25%        1.127680
50%        1.127920
75%        1.128340
max        1.130150
Name: 4, dtype: float64


In [30]:
#predict new interval
save_path='agModels-predict_new_interval'
predictor = TabularPredictor(label=label_new_interval, path=save_path)
predictor.fit(train_data)#, presets='best_quality')
#.fit(presets='best_quality) is slower

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predict_new_interval/"
AutoGluon Version:  0.3.1
Train Data Rows:    500
Train Data Columns: 3
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1.13015, 1.1273600000000001, 1.12807, 0.00058)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11965.48 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeat

[1000]	train_set's rmse: 0.000123863	valid_set's rmse: 0.000161375
[2000]	train_set's rmse: 0.000102504	valid_set's rmse: 0.000144009
[3000]	train_set's rmse: 9.04676e-05	valid_set's rmse: 0.000137154
[4000]	train_set's rmse: 8.11473e-05	valid_set's rmse: 0.000133506
[5000]	train_set's rmse: 7.46757e-05	valid_set's rmse: 0.000130737
[6000]	train_set's rmse: 6.95735e-05	valid_set's rmse: 0.000129526
[7000]	train_set's rmse: 6.5618e-05	valid_set's rmse: 0.000128744
[8000]	train_set's rmse: 6.20708e-05	valid_set's rmse: 0.000128383
[9000]	train_set's rmse: 5.91102e-05	valid_set's rmse: 0.000128124
[10000]	train_set's rmse: 5.637e-05	valid_set's rmse: 0.000127683


	-0.0001	 = Validation score   (root_mean_squared_error)
	3.25s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: LightGBM ...
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 4.03641e-05	valid_set's rmse: 0.000122361


	-0.0001	 = Validation score   (root_mean_squared_error)
	0.65s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.0001	 = Validation score   (root_mean_squared_error)
	0.83s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-0.0001	 = Validation score   (root_mean_squared_error)
	1.67s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.0001	 = Validation score   (root_mean_squared_error)
	0.66s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.0002	 = Validation score   (root_mean_squared_error)
	7.77s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-0.0002	 = Validation score   (root_mean_squared_error)
	0.21s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-0.0003	 = Validation score   (root_mean_squared_error)
	10.83s	 = Training   runtime
	0.15s	 = Validation runtime
Fitti

In [31]:
#contrast leaderboard result of train and test data
predictor.leaderboard(train_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-0.000045,-0.000096,0.016164,0.001978,1.666761,0.016164,0.001978,1.666761,1,True,6
1,KNeighborsDist,-0.000047,-0.000105,0.104927,0.107200,0.014691,0.104927,0.107200,0.014691,1,True,2
2,WeightedEnsemble_L2,-0.000047,-0.000092,0.206047,0.119416,3.634016,0.005278,0.005171,0.393943,2,True,12
3,LightGBMLarge,-0.000050,-0.000110,0.065625,0.009242,0.916664,0.065625,0.009242,0.916664,1,True,11
4,ExtraTreesMSE,-0.000055,-0.000095,0.118980,0.103026,0.656648,0.118980,0.103026,0.656648,1,True,7
5,LightGBM,-0.000063,-0.000122,0.064509,0.011138,0.651084,0.064509,0.011138,0.651084,1,True,4
6,RandomForestMSE,-0.000066,-0.000113,0.117938,0.103072,0.832699,0.117938,0.103072,0.832699,1,True,5
7,LightGBMXT,-0.000076,-0.000128,0.333374,0.046583,3.248820,0.333374,0.046583,3.248820,1,True,3
8,KNeighborsUnif,-0.000114,-0.000137,0.104174,0.104908,0.057574,0.104174,0.104908,0.057574,1,True,1
9,NeuralNetFastAI,-0.000211,-0.000234,0.036751,0.017353,7.772794,0.036751,0.017353,7.772794,1,True,8


In [72]:
#predict close
df_test['pred_col'] = "A"
test_data = TabularDataset(df_test)
y_test = test_data[pred_col]
test_data_nolab = test_data.drop(columns=[pred_col])
test_data_nolab 

NameError: ignored

In [56]:
#predict new interval
predictor = TabularPredictor.load(save_path)
y_pred = predictor.predict(df_test)
print("Predictions: \n", y_pred)
perf = predictor.evaluate_predictions(y_true = y_test, y_pred = y_pred, auxiliary_metrics=True)

KeyError: ignored

In [ ]:
#predict high
label_h = 'eu_High_next_day1' 
print("Summary of class variable: \n", train_data[label_h].describe())

Summary of class variable: 
 count    7999.000000
mean        1.211130
std         0.145835
min         0.842400
25%         1.120930
50%         1.208400
75%         1.317115
max         1.603800
Name: eu_High_next_day1, dtype: float64


In [ ]:
#predict high
save_path='agModels-predictHigh'
predictor = TabularPredictor(label=label_h, path=save_path).fit(train_data)#, presets='best_quality')
#.fit(presets='best_quality) is slower

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictHigh/"
AutoGluon Version:  0.3.1
Train Data Rows:    7999
Train Data Columns: 95
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1.6038, 0.8424, 1.21113, 0.14583)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12148.22 MB
	Train Data (Original)  Memory Usage: 6.08 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Sta

[1000]	train_set's rmse: 0.0028212	valid_set's rmse: 0.00377946
[2000]	train_set's rmse: 0.00201432	valid_set's rmse: 0.00328757
[3000]	train_set's rmse: 0.0015789	valid_set's rmse: 0.00306399
[4000]	train_set's rmse: 0.00129402	valid_set's rmse: 0.00294233
[5000]	train_set's rmse: 0.00108418	valid_set's rmse: 0.00286708
[6000]	train_set's rmse: 0.000921541	valid_set's rmse: 0.0028172
[7000]	train_set's rmse: 0.000792987	valid_set's rmse: 0.00277953
[8000]	train_set's rmse: 0.000686421	valid_set's rmse: 0.00275347
[9000]	train_set's rmse: 0.000599439	valid_set's rmse: 0.00273434
[10000]	train_set's rmse: 0.000526637	valid_set's rmse: 0.00272124


	-0.0027	 = Validation score   (root_mean_squared_error)
	43.93s	 = Training   runtime
	0.94s	 = Validation runtime
Fitting model: LightGBM ...
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1000]	train_set's rmse: 0.000802637	valid_set's rmse: 0.00237549
[2000]	train_set's rmse: 0.000332527	valid_set's rmse: 0.00225848
[3000]	train_set's rmse: 0.000156893	valid_set's rmse: 0.00223788
[4000]	train_set's rmse: 7.84333e-05	valid_set's rmse: 0.00223064
[5000]	train_set's rmse: 3.99913e-05	valid_set's rmse: 0.00222866
[6000]	train_set's rmse: 2.08656e-05	valid_set's rmse: 0.00222765
[7000]	train_set's rmse: 1.09057e-05	valid_set's rmse: 0.00222698
[8000]	train_set's rmse: 5.7962e-06	valid_set's rmse: 0.00222672
[9000]	train_set's rmse: 3.09505e-06	valid_set's rmse: 0.0022266
[10000]	train_set's rmse: 1.64361e-06	valid_set's rmse: 0.00222652


	-0.0022	 = Validation score   (root_mean_squared_error)
	93.18s	 = Training   runtime
	0.71s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.0019	 = Validation score   (root_mean_squared_error)
	87.99s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: CatBoost ...
	-0.0027	 = Validation score   (root_mean_squared_error)
	284.6s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.0019	 = Validation score   (root_mean_squared_error)
	16.05s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: NeuralNetFastAI ...


In [ ]:
#contrast leaderboard result of train and test data
predictor.leaderboard(train_data, silent=True)

In [ ]:
#predict high
test_data = TabularDataset(df_m_5_test_data)
y_test = test_data[label_h]
test_data_nolab = test_data.drop(columns=[label_h])
test_data_nolab.tail()

In [ ]:
#first run to predict eu_High_next_day1
predictor = TabularPredictor.load(save_path)
y_pred = predictor.predict(test_data_nolab)
print("Predictions: \n", y_pred) #try print more rows
perf = predictor.evaluate_predictions(y_true = y_test, y_pred = y_pred, auxiliary_metrics=True)

In [ ]:
#predict low
label_l = 'eu_Low_next_day1' 
print("Summary of class variable: \n", train_data[label_l].describe())

In [ ]:
#predict low
save_path='agModels-predictLow'
predictor = TabularPredictor(label=label_l, path=save_path).fit(train_data)#, presets='best_quality')
#.fit(presets='best_quality) is slower

In [ ]:
#contrast leaderboard result of train and test data
predictor.leaderboard(train_data, silent=True)

In [ ]:
#predict low
test_data = TabularDataset(df_m_5_test_data)
y_test = test_data[label_l]
test_data_nolab = test_data.drop(columns=[label_l])
test_data_nolab.tail()

In [ ]:
#predict eu_Low_next_day1
predictor = TabularPredictor.load(save_path)
y_pred = predictor.predict(test_data_nolab)
print("Predictions: \n", y_pred)
perf = predictor.evaluate_predictions(y_true = y_test, y_pred = y_pred, auxiliary_metrics=True)